In [ ]:
# The code was removed by Watson Studio for sharing.

## Segmenting and Clustering neighborhoods in Toronto <br>

### Part 1 - Setting up neighborhoods data into a pandas dataframe 

In [1]:
# Load data from wikipedia page

import numpy as np
import pandas as pd
d = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df_toronto = d[0]       # First element of the list is the required dataframe 
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [2]:
# Check for not assigned rows
print("Number of not assigned neighbourhoods: ", len(df_toronto[df_toronto['Neighbourhood'] =='Not assigned']))
print("Borough value for such rows:", df_toronto[df_toronto['Neighbourhood'] =='Not assigned']['Borough'].value_counts())

Number of not assigned neighbourhoods:  77
Borough value for such rows: Not assigned    77
Name: Borough, dtype: int64


Hence we see that wherever neighborhood is not assigned, borough is also not assigned. So we drop these rows.

In [3]:
df_toronto.drop(df_toronto[df_toronto['Neighbourhood']=='Not assigned'].index, axis=0, inplace=True)
df_toronto.reset_index(drop=True, inplace=True)

In [4]:
# Merge rows by postal code
df_toronto = df_toronto.groupby('Postcode', as_index=False).aggregate({'Borough': 'first', 'Neighbourhood':', '.join})
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
# Display random rows from the dataframe
x = np.random.randint(0, len(df_toronto), 15)
df_toronto.loc[x,:]

,Postcode,Borough,Neighbourhood
82,M6P,West Toronto,"High Park, The Junction South"
91,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
4,M1H,Scarborough,Cedarbrae
0,M1B,Scarborough,"Rouge, Malvern"
38,M4G,East York,Leaside
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
44,M4N,Central Toronto,Lawrence Park
28,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights"
74,M6E,York,Caledonia-Fairbanks


In [6]:
print("Number or postcodes/ rows:", df_toronto.shape[0])

Number or postcodes/ rows: 103


<br>
<br>

### Part 2 - Add latitude and longitude coordinates of neighborhoods <br>

In [7]:
df_ll = pd.read_csv('http://cocl.us/Geospatial_data')  # Get coordinates from the url
df_ll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_toronto = pd.merge(df_toronto, df_ll, on='Postcode')
df_toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<br>
<br>

### Part - 3 Exploring and clustering the neighborhoods in Toronto 

<br>

In [8]:
# Install dependencies

!pip install geopy
!pip install folium

     |████████████████████████████████| 92kB 19.8MB/s eta 0:00:01


In [9]:
import json
from pandas.io.json import json_normalize  # transform json file into pandas dataframe
import requests

In [10]:
# Take a look at number of boroughs and neighbourhoods
print('Toronto has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

Toronto has 10 boroughs and 103 neighborhoods.


In [11]:
# Let's check how many postcodes belong to each borough
df_toronto['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

<br>
Now let us visualize the ten boroughs on a map. For this I approximate the location of each borough as the average value of the coordinates for all its neighborhoods. 

In [12]:
df_map = df_toronto.drop('Neighbourhood', axis=1).groupby('Borough', as_index=False).mean()

In [13]:
from geopy.geocoders import Nominatim
import folium

In [14]:
# Get coordinates of Toronto

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ttt")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
# Make a map of toronto showing all the boroughs
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, long, borough in zip(df_map['Latitude'], df_map['Longitude'], df_map['Borough']):
    folium.Marker(
    location=[lat, long],
    popup=borough).add_to(map_toronto)
    
map_toronto

<br> 
### Okay now we can start exploring. Here I am picking only the 4 boroughs which contain the word toronto. We will explore all the restaurants in these neighborhoods. Then later on we will cluster the neighborhoods by restaurant frequency, average rating and how expensive/ costly the restaurants are in that neighborhood.
<br>

In [16]:
# The code was removed by Watson Studio for sharing.

In [18]:
# First let us make a subset of the original dataframe containing all the boroughs having the name Toronto
df_new = df_toronto[df_toronto['Borough'].str.contains("Toronto")]
df_new.reset_index(drop=True, inplace=True)
print("Shape of new dataframe", df_new.shape)
df_new.head()

Shape of new dataframe (39, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [19]:
# Define a function to make API calls and get the relevant information for all restaurants

def get_venues_table(postcodes, boroughs, neighborhoods, latitudes, longitudes, query, radius=500):
    
    venues_list=[]
    for pc, bor, n, lat, long in zip(postcodes, boroughs, neighborhoods, latitudes, longitudes):
        
        url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, long, query, radius)
        
        result1 = requests.get(url1).json()['response']['venues']
        
        for v in result1:
            
            v_list = []
            try:
                v_list = v_list + [pc, bor, n, v['name'], v['location']['address'], v['location']['postalCode']]
            except:
                v_list = v_list + [pc, bor, n, v['name'], "N/A", "N/A"]
    
            url2 = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(
                v['id'], CLIENT_ID, CLIENT_SECRET, VERSION)
            result2 = requests.get(url2).json()['response']['venue']
            
            try:
                v_list = v_list + [result2['rating'], result2['price']['tier'], v['location']['lat'], v['location']['lng']]
            except:
                try:
                    v_list = v_list + ["N/A", result2['price']['tier'], v['location']['lat'], v['location']['lng']]
                except:
                    try:
                        v_list = v_list + [result2['rating'], "N/A", v['location']['lat'], v['location']['lng']]
                    except:
                        v_list = v_list + ["N/A", "N/A", v['location']['lat'], v['location']['lng']]
            
            venues_list.append(v_list)
    
    return venues_list


In [12]:
# Create the restaurants dataframe

df_restaurants = pd.DataFrame(data = get_venues_table(df_new.loc[0:3, 'Postcode'], df_new.loc[0:3, 'Borough'], df_new.loc[0:3, 'Neighbourhood'],
                    df_new.loc[0:3, 'Latitude'], df_new.loc[0:3, 'Longitude'], "Restaurant"), 
                       columns = ['Postcode', 'Borough', 'Neigborhood', 'Restarant', 'Address', 'Rating', 'Price Tier', 'Latitude', 'Longitude'])
df_restaurants.head(20)

,Postcode,Borough,Neigborhood,Restarant,Address,Rating,Price Tier,Latitude,Longitude
0,M4E,East Toronto,The Beaches,Seaspray Restaurant,629 Kingston Rd,N/A,1,43.678888,-79.298167
1,M4K,East Toronto,"The Danforth West, Riverdale",Herby Restaurant,397 Danforth Ave,N/A,N/A,43.677450,-79.351290
2,M4K,East Toronto,"The Danforth West, Riverdale",Beiteddine Lebanese Restaurant,N/A,N/A,N/A,43.678065,-79.348542
3,M4K,East Toronto,"The Danforth West, Riverdale",Osmow’s Authentic Mediterranean Restaurant,497 Danforth Avenue,N/A,2,43.677883,-79.349250
4,M4K,East Toronto,"The Danforth West, Riverdale",Megas Restaurant,402 Danforth Ave,6.5,2,43.677692,-79.351522
5,M4K,East Toronto,"The Danforth West, Riverdale",Florentina's Italian Restaurant,N/A,N/A,2,43.676562,-79.355699
6,M4K,East Toronto,"The Danforth West, Riverdale",Katsu Japanese Restaurant,572 Danforth Ave,6.9,2,43.678619,-79.347024
7,M4K,East Toronto,"The Danforth West, Riverdale",The Palace Restaurant,722 Pape Avenue,N/A,2,43.679827,-79.345331
8,M4K,East Toronto,"The Danforth West, Riverdale",Simone's Caribbean Restaurant,596 Danforth Avenue,7.6,2,43.678655,-79.346582
9,M4K,East Toronto,"The Danforth West, Riverdale",Friendly Greek Restaurant,494 Danforth Ave.,6.6,2,43.678428,-79.347642


In [17]:
df_restaurants.shape # Check how many restaurants were returned across the 4 boroughs

(408, 9)

In [24]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'toronto_restaurants.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'datasciencecapstone-donotdelete-pr-9xufyyyz2be1rz',
 'asset_id': '132a9347-6bcf-4ab6-b638-0a555aa475b8'}

The search returned 408 restaurants. That's too many. Probably it is because we chose a large radius, 500m. Hence searches often criss-crossed across neighborhood and resulted in many duplicates. Let's check for duplicates.

In [38]:
df_restaurants[df_restaurants.duplicated(subset=['Restarant', 'Latitude', 'Longitude'])].sort_values(by='Restarant').head(30)

,Postcode,Borough,Neigborhood,Restarant,Address,Rating,Price Tier,Latitude,Longitude
195,M5H,Downtown Toronto,"Adelaide, King, Richmond",A&W,1 Richmond Street West,5.6,1,43.651378,-79.378986
270,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",A&W,1 Richmond Street West,5.6,1,43.651378,-79.378986
402,M7A,Downtown Toronto,Queen's Park,A&W,496 Yonge Street,6,1,43.663094,-79.383971
241,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",A&W,1 Richmond Street West,5.6,1,43.651378,-79.378986
363,M5X,Downtown Toronto,"First Canadian Place, Underground city",A&W,1 Richmond Street West,5.6,1,43.651378,-79.378986
142,M5G,Downtown Toronto,Central Bay Street,Adega Restaurant,33 Elm St,7.2,3,43.657519,-79.383462
163,M5G,Downtown Toronto,Central Bay Street,Akashiro Japanese Restaurant & Bar,220 Yonge St.,5.6,2,43.655965,-79.380541
160,M5G,Downtown Toronto,Central Bay Street,Alio Restaurant & Wine Bar,108 Dundas St W,5.2,4,43.655655,-79.384124
188,M5H,Downtown Toronto,"Adelaide, King, Richmond",Anoush Restaurant,250 Dundas St W,N/A,2,43.654588,-79.389692
341,M5X,Downtown Toronto,"First Canadian Place, Underground city",Azure Restaurant & Bar,225 Front St W,5.9,4,43.644749,-79.385113


<br>
<br>

In [20]:
# The code was removed by Watson Studio for sharing.

,Postcode,Borough,Neigborhood,Restarant,Address,Rating,Price Tier,Latitude,Longitude
0,M4E,East Toronto,The Beaches,Seaspray Restaurant,629 Kingston Rd,NaN,1.0,43.678888,-79.298167
1,M4K,East Toronto,"The Danforth West, Riverdale",Herby Restaurant,397 Danforth Ave,NaN,NaN,43.677450,-79.351290
2,M4K,East Toronto,"The Danforth West, Riverdale",Beiteddine Lebanese Restaurant,NaN,NaN,NaN,43.678065,-79.348542
3,M4K,East Toronto,"The Danforth West, Riverdale",Osmow’s Authentic Mediterranean Restaurant,497 Danforth Avenue,NaN,2.0,43.677883,-79.349250
4,M4K,East Toronto,"The Danforth West, Riverdale",Megas Restaurant,402 Danforth Ave,6.5,2.0,43.677692,-79.351522


In [21]:
# Delete the duplicate values from the restaurants dataframe. Only those entries remain where restaurant is in the correct neighborhood.
# Note df_rest is the restaurants dataframe

df_rest.rename(columns = {'Restarant':'Restaurant', 'Latitude':'lat_R', 'Longitude':'long_R'}, inplace=True)
df_rest = df_rest.merge(df_new[['Postcode', 'Latitude', 'Longitude']], on='Postcode')
df_rest['Distance_sq'] = np.square((df_rest['lat_R'] - df_rest['Latitude'])) + np.square((df_rest['long_R'] - df_rest['Longitude']))
df_rest.sort_values(by=['Restaurant', 'Distance_sq'], axis=0, inplace=True)
df_rest.drop_duplicates(subset=['Restaurant', 'lat_R', 'long_R'], keep='first', inplace=True)
df_rest.sort_values(by=['Postcode'], inplace=True)

df_rest.drop(columns=['Latitude', 'Longitude', 'Distance_sq'], inplace=True)
df_rest.reset_index(drop=True, inplace=True)

In [22]:
df_rest.head()

,Postcode,Borough,Neigborhood,Restaurant,Address,Rating,Price Tier,lat_R,long_R
0,M4E,East Toronto,The Beaches,Seaspray Restaurant,629 Kingston Rd,NaN,1.0,43.678888,-79.298167
1,M4K,East Toronto,"The Danforth West, Riverdale",Friendly Greek Restaurant,494 Danforth Ave.,6.6,2.0,43.678428,-79.347642
2,M4K,East Toronto,"The Danforth West, Riverdale",Megas Restaurant,402 Danforth Ave,6.5,2.0,43.677692,-79.351522
3,M4K,East Toronto,"The Danforth West, Riverdale",Osmow’s Authentic Mediterranean Restaurant,497 Danforth Avenue,NaN,2.0,43.677883,-79.349250
4,M4K,East Toronto,"The Danforth West, Riverdale",Pantheon,407 Danforth Ave.,8.7,2.0,43.677621,-79.351434


In [23]:
print("Ok, so there are a total of {} restaurnats in Toronto".format(len(df_rest)))

Ok, so there are a total of 234 restaurnats in Toronto


In [25]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ttt")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
# Now plot all the restaurants on a map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, long, label in zip(df_rest['lat_R'], df_rest['long_R'], df_rest['Restaurant']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<br>
So, now let us get to the clustering part. We have to cluster the 39 neighborhoods in 4 boroughs in Toronto. The parameters that we will use for clustering are number of restaurants, average rating of restaurants, price tier/ how costly the restaurants are in a given neighborhood. As the number of parameters are less, I will only be dividing the neighborhoods into 3 clusters.
<br>

In [27]:
# Set up new dataframe for clustering

df_rest2 = df_rest.drop(columns=['Address', 'lat_R', 'long_R'])
df_rest2 = df_rest2.groupby('Postcode', as_index=False).aggregate(
    {'Borough':'first', 'Neigborhood':'first', 'Restaurant':'count', 'Rating':'mean', 'Price Tier':'mean'})
df_rest2.dropna(inplace=True)  # Drop nan values
df_rest2.reset_index(drop=True, inplace=True)

In [28]:
df_rest2.head()

,Postcode,Borough,Neigborhood,Restaurant,Rating,Price Tier
0,M4K,East Toronto,"The Danforth West, Riverdale",11,7.26,1.888889
1,M4L,East Toronto,"The Beaches West, India Bazaar",4,6.00,1.750000
2,M4P,Central Toronto,Davisville North,4,7.10,1.750000
3,M4R,Central Toronto,North Toronto West,2,6.60,2.000000
4,M4S,Central Toronto,Davisville,5,6.30,1.500000


In [29]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [30]:
X = df_rest2.values[:,3:6]
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[ 0.35648667,  0.50650807, -0.04254172],
       [-0.72485623, -1.6849412 , -0.4190453 ],
       [-0.72485623,  0.2282288 , -0.4190453 ],
       [-1.03381134, -0.64139393,  0.25866115],
       [-0.57037867, -1.16316757, -1.09675174],
       [-0.72485623,  1.79354971,  1.16226974],
       [-0.106946  , -0.69936878, -1.29038216],
       [ 1.12887445, -0.88985757, -0.28350401],
       [-0.41590111,  0.92392698, -0.64494745],
       [ 0.66544178, -0.21900575,  0.05013609],
       [ 0.97439689,  0.59540284,  1.0331828 ],
       [ 2.21021735, -0.53436344, -0.02668894],
       [ 0.20200911, -0.29354484,  1.16226974],
       [ 0.51096423,  0.94877334,  0.4845633 ],
       [-0.41590111, -0.93126817,  1.16226974],
       [ 0.35648667, -0.66624029,  0.50509985],
       [-0.72485623, -1.27911726, -1.77445819],
       [ 3.29156024, -0.23556999, -0.86305987],
       [ 0.04753156,  0.150929  , -0.04254172],
       [-0.87933378,  1.53266289,  2.06587834],
       [-0.87933378,  0.75000243,  1.162

In [31]:
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[0 1 1 1 1 0 1 2 0 0 0 2 0 0 0 0 1 2 0 0 0 0 1 0 1 1]


In [32]:
# add Labels to the original dataframe
df_rest2['Label'] = labels

In [33]:
df_rest2

,Postcode,Borough,Neigborhood,Restaurant,Rating,Price Tier,Label
0,M4K,East Toronto,"The Danforth West, Riverdale",11,7.260000,1.888889,0
1,M4L,East Toronto,"The Beaches West, India Bazaar",4,6.000000,1.750000,1
2,M4P,Central Toronto,Davisville North,4,7.100000,1.750000,1
3,M4R,Central Toronto,North Toronto West,2,6.600000,2.000000,1
4,M4S,Central Toronto,Davisville,5,6.300000,1.500000,1
5,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",4,8.000000,2.333333,0
6,M4X,Downtown Toronto,"Cabbagetown, St. James Town",8,6.566667,1.428571,1
7,M4Y,Downtown Toronto,Church and Wellesley,16,6.457143,1.800000,2
8,M5A,Downtown Toronto,Harbourfront,6,7.500000,1.666667,0
9,M5B,Downtown Toronto,"Ryerson, Garden District",13,6.842857,1.923077,0


In [34]:
df_rest2[['Restaurant', 'Rating', 'Price Tier', 'Label']].groupby('Label').mean()

,Restaurant,Rating,Price Tier
Label,,,
0,8.214286,7.319252,2.139463
1,4.666667,6.529630,1.587302
2,23.000000,6.650672,1.760315


Interpretation of results - <br>
label 0: Neighbourhoods with expensive restaurants and moderate number of eating options.                          
label 1: Neighbourhoods with cheap restaurants and low number of eating options. <br>
label 2: Neighbourhoods with cheap restaurants and high number of eating options.<br>
On the whole we see that expensive restaurants class has a higher rating.
<br>

In [35]:
# Merge original dataframe with labels
df_new2 = df_new.merge(df_rest2[['Postcode', 'Label']], on='Postcode')

In [36]:
mark_color = df_new2['Label'].replace({0: 'purple', 1:'red', 2:'blue'})

In [37]:
# Let's visualize the clusters on a map
map_cluster = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, long, n, c in zip(df_new2['Latitude'], df_new2['Longitude'], df_new2['Neighbourhood'], mark_color):
    label = folium.Popup(n, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=7,
        popup=label,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity=0.6,
        parse_html=False).add_to(map_cluster)  
    
map_cluster

<br>
Blue: high density, Purple: expensive. Thus we can see that neighborhoods in Downtown Toronto have expensive restaurants and a good number of eating options. As we move inland into neighborhoods in Central Toronto there is less density of restaurants and they are cheaper. 